## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [1]:
from datetime import datetime
from scipy import stats  
import os
import pytz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  

%matplotlib inline

### Creating IDX Equities Time Series

In [7]:
idx_combined = pd.read_csv('/home/nikki/documents/amibroker/idx_exported_combined.csv', parse_dates=[1])

In [2]:
# Location of the IDX Stocks OHLC csv files
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv'
# csv_loc = '/Users/nikki/Documents/idx_exported_csv'
csv_loc = '/home/nikki/documents/amibroker/exported-csv/'

# Backup if the first one doesn't work
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv_gama'

tickers = os.listdir(csv_loc)

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for ticker in tickers:
    data_idx[ticker[:-4]] = pd.read_csv(f'{csv_loc}/{ticker}',
                                        index_col='date',
                                        parse_dates=['date'])

In [4]:
data_idx['ASII'].tail()

,open,high,low,close,volume
date,,,,,
2020-08-31,5325.0,5350.0,5100.0,5100.0,42240600
2020-09-01,5175.0,5250.0,5100.0,5250.0,20881000
2020-09-02,5300.0,5350.0,5200.0,5300.0,19074500
2020-09-03,5350.0,5350.0,5125.0,5175.0,38323300
2020-09-04,5100.0,5150.0,5025.0,5050.0,47966200


### Stocks on the Move


In [5]:
def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

In [6]:
def volatility(ts, period=20):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [12]:
# How many (series) candles back for momentum calculation?
momentum_window = 96

# Create an empty DataFrame to store score
momentum_table = pd.DataFrame(columns=['ticker', 'score', 'vola', 'inv_vola'])

# How many (series) candles back for std dev calculation?
vola_window = 20

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_idx.items():
    # Need the stocks to exist at least 3 years prior (756 trading days)
    if len(timeseries) < 756:
        continue
        
    # If there's a 0 volume in the stocks, drop it 
    if 0 in timeseries['volume'].iloc[-momentum_window:].tolist():
        continue

    momentum_series = timeseries['close'].iloc[-momentum_window:]
    score = momentum_score(momentum_series)
    vola_series = timeseries['close']
    vola = volatility(vola_series, vola_window)
    avg_volume = timeseries['volume'].rolling(vola_window).mean().iloc[-1] / 100
    
    if avg_volume < 500:
        continue

    momentum_table = momentum_table.append({'ticker': ticker,
                                            'score': score,
                                            'vola': vola,
                                            'avg_vol': avg_volume},
                                           ignore_index=True)
    
    momentum_table['inv_vola'] = 1 / momentum_table['vola']

In [13]:
momentum_table.sort_values('score', ascending=False)[:100].to_clipboard()

In [9]:
momentum_table[momentum_table['ticker'] == 'AKSI']

,ticker,score,vola,inv_vola,avg_vol
12,AKSI,-0.002968,0.047425,21.086041,825.55


In [9]:
momentum_table.sort_values('score', ascending=False)[:30]

,ticker,score,vola,inv_vola,avg_vol
14,WIIM,4162.671908,0.042206,23.693526,668082.750
188,AGRO,3288.260306,0.048225,20.736251,1839095.276
122,CENT,2487.975376,0.060895,16.421704,258450.198
32,MARK,2356.605924,0.027323,36.598694,72904.000
86,KRAS,1540.047676,0.022295,44.853335,917795.208
141,DOID,1025.617685,0.044359,22.543477,1317032.332
79,NIKL,904.989937,0.038694,25.843524,104038.850
145,SMDR,799.552955,0.038892,25.712239,146819.548
108,IMJS,719.663460,0.074958,13.340731,462929.852
85,SMBR,617.603444,0.051120,19.561909,567661.692


In [10]:
momentum_table.sort_values('score', ascending=False)[30:50]

,ticker,score,vola,inv_vola,avg_vol
21,PTRO,225.927561,0.018699,53.478241,14820.300
109,BNII,211.639909,0.039131,25.555319,195759.398
165,INCO,205.520434,0.019923,50.193673,144604.250
52,MDKA,205.463693,0.031363,31.884485,1375980.348
89,TKIM,197.257865,0.021971,45.515323,132050.600
217,ADES,194.964266,0.028279,35.361699,2382.450
67,BMRI,186.295963,0.021418,46.688772,506511.100
54,LSIP,179.183145,0.018393,54.368642,453676.756
201,CPIN,174.288533,0.012622,79.224323,89611.150
35,ERAA,171.742505,0.025366,39.422616,223138.950


In [11]:
momentum_table.sort_values('score', ascending=False)[50:100]

,ticker,score,vola,inv_vola,avg_vol
149,AKRA,131.638805,0.020255,49.370590,226404.800
208,IPOL,122.141408,0.080315,12.451013,9537.500
150,CEKA,118.632404,0.020238,49.410884,2032.750
210,DKFT,117.621931,0.034017,29.397030,125445.700
87,LPCK,111.867498,0.037177,26.898226,21642.450
113,DSNG,107.900255,0.034090,29.334298,82069.450
25,MBSS,107.281962,0.017006,58.801080,16421.150
4,SMGR,105.445006,0.023827,41.969618,65510.800
178,SDPC,105.347025,0.043572,22.950388,3493.850
204,BTPN,103.324598,0.015763,63.439793,1545.400
